In [36]:
import pandas as pd

In [37]:
# loading the data
user_reviews_path = "C:/Users/vanee/OneDrive/Documents/AI/UserReviews.xlsx"
expert_reviews_path = "C:/Users/vanee/OneDrive/Documents/AI/ExpertReviews.xlsx"
meta_clean_path = "C:/Users/vanee/OneDrive/Documents/AI/metaClean43Brightspace.xlsx"
sales_path = "C:/Users/vanee/OneDrive/Documents/AI/sales.xlsx"

# reading the files
user_reviews = pd.read_excel(user_reviews_path)
#expert_reviews = pd.read_excel(expert_reviews_path)
#meta_clean = pd.read_excel(meta_clean_path)
#sales = pd.read_excel(sales_path)

In [38]:
# check the first few rows of user reviews
user_reviews.head()

,url,idvscore,reviewer,dateP,Rev,thumbsUp,thumbsTot
0,https://www.metacritic.com/movie/bronson,8,'Longbottom94',"'Apr 25, 2013'",'Many have dismissed this film for not explor...,2,2
1,https://www.metacritic.com/movie/bronson,9,'MartinB',"'Oct 13, 2009'",'Anyone who doesn t like this movie simply ju...,0,1
2,https://www.metacritic.com/movie/bronson,10,'Jaakko',"'Jul 19, 2012'",'Not sure what to think at this film at first...,1,1
3,https://www.metacritic.com/movie/bronson,6,'CapoR',"'Oct 13, 2009'",'Nicely portrayed but it lacks the elements t...,0,1
4,https://www.metacritic.com/movie/bronson,8,'OrwellB.',"'Oct 10, 2009'",'Bronson is more than entertainment. It is ar...,0,0


In [39]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319662 entries, 0 to 319661
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   url        319662 non-null  object
 1   idvscore   316258 non-null  object
 2   reviewer   316255 non-null  object
 3   dateP      316249 non-null  object
 4   Rev        316249 non-null  object
 5   thumbsUp   316082 non-null  object
 6   thumbsTot  316086 non-null  object
dtypes: object(7)
memory usage: 17.1+ MB


In [40]:
# summary statistics of user reviews
user_reviews.describe()

,url,idvscore,reviewer,dateP,Rev,thumbsUp,thumbsTot
count,319662,316258,316255,316249,316249,316082,316086
unique,11364,16,106795,6489,311558,209,287
top,https://www.metacritic.com/movie/joker,10,'TVJerry',"'Mar 8, 2019'",'',0,0
freq,2764,74562,1626,878,773,197106,147659


In [41]:
# check for missing values user reviews
user_reviews.isnull().sum()

url             0
idvscore     3404
reviewer     3407
dateP        3413
Rev          3413
thumbsUp     3580
thumbsTot    3576
dtype: int64

In [42]:
user_reviews['url'].nunique()

11364

In [43]:
#user_reviews['idvscore'].fillna(user_reviews['idvscore'].median(), inplace=True)

# checking non-numeric values for idvscore column
non_numeric_values = user_reviews[~user_reviews['idvscore'].apply(pd.to_numeric, errors='coerce').notnull()]
print(non_numeric_values)

                                                      url idvscore reviewer  \
29       https://www.metacritic.com/movie/a-screaming-man      NaN      NaN   
30        https://www.metacritic.com/movie/lovers-of-hate      NaN      NaN   
34            https://www.metacritic.com/movie/viva-riva!      NaN      NaN   
130         https://www.metacritic.com/movie/september-11      NaN      NaN   
134         https://www.metacritic.com/movie/catching-out      NaN      NaN   
...                                                   ...      ...      ...   
318596  https://www.metacritic.com/movie/domestic-viol...      NaN      NaN   
318841   https://www.metacritic.com/movie/giorni-e-nuvole      NaN      NaN   
318842  https://www.metacritic.com/movie/the-last-mist...      NaN      NaN   
319108             https://www.metacritic.com/movie/iousa      NaN      NaN   
319139      https://www.metacritic.com/movie/frontrunners      NaN      NaN   

       dateP  Rev thumbsUp thumbsTot  
29       NaN

In [44]:
# drop columns with NaN values for both idvscore & reviewer
user_reviews.dropna(subset=['idvscore', 'reviewer'], inplace=True)

# checking for remaining null values
print(user_reviews.isnull().sum())

url            0
idvscore       0
reviewer       0
dateP          5
Rev            5
thumbsUp     173
thumbsTot    172
dtype: int64


In [46]:
# replacing NaN values with 0
user_reviews['thumbsUp'] = user_reviews['thumbsUp'].fillna(0)
user_reviews['thumbsTot'] = user_reviews['thumbsTot'].fillna(0)

# replace missing Rev values
user_reviews['Rev'] = user_reviews['Rev'].fillna('No review provided')

# replace missing dateP values
user_reviews['dateP'] = user_reviews['dateP'].fillna('Unknown')

# checking for null values again
user_reviews.isnull().sum()

C:\Users\vanee\AppData\Local\Temp\ipykernel_33740\4001281752.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  user_reviews['thumbsUp'] = user_reviews['thumbsUp'].fillna(0)


url          0
idvscore     0
reviewer     0
dateP        0
Rev          0
thumbsUp     0
thumbsTot    0
dtype: int64

In [47]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 316254 entries, 0 to 319661
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   url        316254 non-null  object
 1   idvscore   316254 non-null  object
 2   reviewer   316254 non-null  object
 3   dateP      316254 non-null  object
 4   Rev        316254 non-null  object
 5   thumbsUp   316254 non-null  int64 
 6   thumbsTot  316254 non-null  object
dtypes: int64(1), object(6)
memory usage: 19.3+ MB


In [51]:
# converting numeric columns from string to int
user_reviews[['idvscore', 'thumbsTot']] = user_reviews[['idvscore', 'thumbsTot']].fillna(0).astype('int64')

In [52]:
user_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 316254 entries, 0 to 319661
Data columns (total 7 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   url        316254 non-null  object
 1   idvscore   316254 non-null  int64 
 2   reviewer   316254 non-null  object
 3   dateP      316254 non-null  object
 4   Rev        316254 non-null  object
 5   thumbsUp   316254 non-null  int64 
 6   thumbsTot  316254 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 19.3+ MB
